In [148]:
import json
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
from bokeh.models import Span

In [149]:
markers = []
avg_dist = []
beaconNum = ""
all_times = []
initial_time = 0

In [ ]:
def moving_average(window):  # parameter represents window for average moving window filter
    global all_times  # list of all times
    global avg_dist  # list of all distances
    numInWindow = 0
    
    for i in range(len(avg_dist)):  # for every single second 0 thru 399
        for j in range(len(all_times)): # loop through every single time 0 thru len(x)-1
            if all_times[j] == i:  # if time is in proper frame
                numInWindow += 1
                avg_dist[i] = avg_dist[i] + avg_dist[j]
        if numInWindow > 0:
            average_dist[i] = average_dist[i] / numInWindow
        numInWindow = 0

In [150]:
def plot_data(i):
    global markers
    global avg_dist
    global beaconNum
    global all_times
    
    output_notebook()
    p = figure(
        tools="pan, box_zoom, wheel_zoom, reset, resize",
        title=beaconNum, x_axis_label='Time Elapsed (seconds)', y_axis_label='Distance (meters)',
        x_range=(min(all_times), max(all_times)), y_range=(min(avg_dist), max(avg_dist)),
        plot_width=900, plot_height=400
    )
    
    # plot measured average distances
    p.line(
        all_times, avg_dist,
        line_color = "navy", legend="Measured"
    )
    
    '''
    actual = []
    for z in range(9):
        actual.append(z)
    
    # plot actual average distances
    p.line(
        markers, actual,
        line_color = "firebrick", legend="Actual"
    )
    '''
    
    # Vertical line
    for m in range(len(markers)):
        vline = Span(location=markers[m], dimension='height', line_color='firebrick',
                     line_width=3, line_alpha=0.4)
        p.renderers.extend([vline])
    
    show(p)

In [151]:
def get_data(i):  # open text file and extract data
    global markers
    global avg_dist
    global beaconNum
    global all_times
    global initial_time
    z = 0
    temp_times = []
    avg_dist[:] = []
    first = True
    
    if i == 0:
        textName = "0_0.txt"
        beaconNum = "Beacon 0:0 Avg Distance"
    elif i == 1:
        textName = "2_16.txt"
        beaconNum = "Beacon 2:16"
    else:
        textName = "3_13.txt"
        beaconNum = "Beacon 3:13"
        
    f = open(textName)

    for line in f:
        words = line.split()
        time = int(words[0])  # get time
        time = (time - initial_time) / 1000  # get elapsed time in seconds
        parsed_json = json.loads(words[1])  # parse json
        rssi = parsed_json['rssi']
        distance = 10**((rssi+60)/-20)  # log-distance pathloss model
        all_times.append(time)
        avg_dist.append(distance)
        '''
        temp_times.append(distance)
        if z < 8 and time >= markers[z+1]:
            avg_dist.append(np.mean(temp_times))
            z += 1  # next meter block
            temp_times[:] = []
            temp_times.append(distance)
    # for meter 9
    avg_dist.append(np.mean(temp_times))
    '''

    f.close()  # close file

In [152]:
def get_markers():
    global markers
    global initial_time
    m = open("markers.txt")
    i = 0
    initial_time = 0
    for line in m:
        words = line.split()
        time = int(words[0])  # get raw time
        markers.append(time)
        if i == 0:
            initial_time = markers[0]
        markers[i] = (markers[i] - initial_time) / 1000
        i += 1
    m.close()

In [153]:
def main():
    global markers
    get_markers()  # get meter marks
    print("markers: ", markers)
    for i in range(3):
        get_data(i)
        plot_data(i)

In [154]:
if __name__ == "__main__":
    main()

markers:  [0.0, 11.895, 25.491, 38.143, 50.902, 64.67, 76.805, 88.973, 100.948]


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...